# Industry 4.0 의 중심, AI - ML&DL

<div align='right'><font size=2 color='gray'>Machine Learning & Deep Learning with TensorFlow @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, 김진수</font></div>
<hr>

# Tip. Saver & Model Reuse
> 모델을 저장하고 재사용  

In [1]:
import time
import os
import sys

def chk_processting_time(start_time, end_time):
    process_time = end_time - start_time
    p_time = int(process_time)
    p_min = p_time // 60
    p_sec = p_time %  60
    print('처리시간 : {p_min}분 {p_sec}초 경과되었습니다.'.format(
            p_min = p_min, 
            p_sec = p_sec
        ))
    return process_time


In [2]:
%%writefile ./data/train_data.csv
0, 0, 1, 0, 0
1, 0, 0, 1, 0
1, 1, 0, 0, 1
0, 0, 1, 0, 0
0, 0, 1, 0, 0
0, 1, 0, 0, 1

Overwriting ./data/train_data.csv


In [3]:
# 학습모델 저장폴더 생성
MODEL_DIR = './models/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

### 학습데이터 로딩
> train_data.csv
- 속성 : 털, 날개 
- 클래스 : 파충류, 포유류, 조류 → One-hot

In [4]:
import tensorflow as tf
import numpy as np

C:\Python\Anaconda3-50\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
data = np.loadtxt('./data/train_data.csv', delimiter=',', unpack=True, dtype='float32')
data

array([[0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.]], dtype=float32)

In [6]:
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [7]:
x_data, y_data

(array([[0., 0.],
        [1., 0.],
        [1., 1.],
        [0., 0.],
        [0., 0.],
        [0., 1.]], dtype=float32), array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.]], dtype=float32))

### 신경망 모델 구성

In [8]:
# 학습에 직접적으로 사용하지 않고 학습 횟수에 따라 단순히 증가시킬 변수를 만듭니다.
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

# global_step로 넘겨준 변수를, 학습용 변수들을 최적화 할 때 마다 학습 횟수를 하나씩 증가시킵니다.
train_op = optimizer.minimize(cost, global_step=global_step)


### 신경망 모델 학습

In [9]:
sess = tf.Session()

# 모델을 저장하고 불러오는 API를 초기화합니다.
# global_variables() 함수를 통해 앞서 정의하였던 변수들을 저장하거나 불러올 변수들로 설정합니다.
saver = tf.train.Saver(tf.global_variables())

chkpoint = tf.train.get_checkpoint_state('./models')
if chkpoint and tf.train.checkpoint_exists(chkpoint.model_checkpoint_path):
    saver.restore(sess, chkpoint.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# 최적화 진행
for step in range(3):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 최적화가 끝난 뒤, 변수를 저장합니다.
saver.save(sess, './models/ani_class.dnn', global_step=global_step)

INFO:tensorflow:Restoring parameters from ./models\test.model-3
Step: 4,  Cost: 1.064
Step: 5,  Cost: 1.023
Step: 6,  Cost: 0.987


'./models/ani_class.dnn-6'

### 결과 확인 
> 0: 파충류, 1: 포유류, 2: 조류 

In [10]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값 :', sess.run(prediction, feed_dict={X: x_data}))
print('실제값 :', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : %.2f %%' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값 : [0 0 2 0 0 2]
실제값 : [0 1 2 0 0 2]
정확도 : 83.33 %


<hr>
<marquee><font size=3 color='brown'>The BigpyCraft find the information to design valuable society with Technology & Craft.</font></marquee>
<div align='right'><font size=2 color='gray'> &lt; The End &gt; </font></div>